In [1]:
from google.colab import files
# 
uploaded=files.upload()
for fn in uploaded.keys():
    print('user uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

Saving zoo.data to zoo.data
Saving zoo.names to zoo.names
user uploaded file "zoo.data" with length 4126 bytes
user uploaded file "zoo.names" with length 2587 bytes


In [3]:
!ls

sample_data  zoo.data  zoo.names


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [7]:
data=pd.read_table(r"zoo.data",header=None,sep=',')
data.shape

(101, 18)

In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [9]:
data.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000
mean,0.425743,0.198020,0.584158,0.405941,0.237624,0.356436,0.554455,0.603960,0.821782,0.792079,0.079208,0.168317,2.841584,0.742574,0.128713,0.435644,2.831683
std,0.496921,0.400495,0.495325,0.493522,0.427750,0.481335,0.499505,0.491512,0.384605,0.407844,0.271410,0.376013,2.033385,0.439397,0.336552,0.498314,2.102709
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,2.000000
75%,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,1.000000,4.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,1.000000,1.000000,1.000000,7.000000


In [0]:
del data[0]

In [0]:
labels=data[17]

In [0]:
del data[17]

In [1]:
def one_hot(raw_labels): ## function for one hot encoding of the label
    import numpy as np
    raw_labels=np.array(raw_labels,np.int32)
    n_values = np.max(raw_labels) + 1
    onehotDash=np.eye(n_values)[raw_labels]
    onehot=np.reshape(onehotDash,(len(raw_labels),8))##we reshape from a 3D matrix as built by default using eye function to a 2D array
    return onehot

In [16]:
one_hot_labels=one_hot(labels)
print(one_hot_labels.shape)

(101, 8)


In [32]:
print(type(one_hot_labels)) ## type of the label now 

<class 'numpy.ndarray'>


In [0]:
h1=50
import tensorflow as tf ## importing tensorflow

x=tf.placeholder(shape=[None,16],dtype=tf.float32) ## placeholder for feature vector
y=tf.placeholder(shape=[None,8],dtype=tf.float32) ## palceholder for lable vector
def forward(data): ## function for forward passs 
    std1=(1/np.sqrt(16))
    w1=tf.Variable(np.random.normal(0,std1,(16,h1))) ## weight initialisation for the input layer to hidden layer 
    b1=tf.Variable(tf.constant(0.2,shape=[h1]))
    w1=tf.cast(w1,tf.float32)
    
    std2=(1/np.sqrt(h1)) ## specifying standard deviation for w initialisation
    w2=tf.Variable(np.random.normal(0,std2,(h1,8)))  ## hidden layer to output layer 
    b2=tf.Variable(tf.constant(0.2,shape=[8]))
    w2=tf.cast(w2,tf.float32)
    
    a0=data
    a1=tf.nn.relu(tf.matmul(a0,w1) +b1) ## applying relu as activation
    a3=(tf.matmul(a1,w2) +b2) ## computing output
    return a3
def train():
    batch=20 ## batch size 20
    pred=forward(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) ## cross entropy 
    train_step = tf.train.AdamOptimizer(0.01).minimize(cost) ## applying adam for optimisation
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) ## computing accuracy
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer()) ## initialising tensors
        for j in range(15):
            for i in range(5):
                print(i)
                x_batch=data.iloc[i*batch:(i+1)*batch,:]
                y_batch=one_hot_labels[i*batch:(i+1)*batch,:]
                sess.run(train_step,feed_dict={x:x_batch,y:y_batch})
                
                

            print('accuracy after epoch is',sess.run(accuracy,feed_dict={x:data,y:one_hot_labels})) ## computing accuracy and loss
            
            print('corresponding loss',sess.run(cost,feed_dict={x:data,y:one_hot_labels}))## after every epoch

In [34]:
train()

0
1
2
3
4
accuracy after epoch is 0.5049505
corresponding loss 1.4595953
0
1
2
3
4
accuracy after epoch is 0.6831683
corresponding loss 1.0642021
0
1
2
3
4
accuracy after epoch is 0.7821782
corresponding loss 0.7107205
0
1
2
3
4
accuracy after epoch is 0.84158415
corresponding loss 0.4747323
0
1
2
3
4
accuracy after epoch is 0.9108911
corresponding loss 0.34249339
0
1
2
3
4
accuracy after epoch is 0.96039605
corresponding loss 0.26290303
0
1
2
3
4
accuracy after epoch is 0.980198
corresponding loss 0.20183234
0
1
2
3
4
accuracy after epoch is 1.0
corresponding loss 0.1578393
0
1
2
3
4
accuracy after epoch is 1.0
corresponding loss 0.12286682
0
1
2
3
4
accuracy after epoch is 0.990099
corresponding loss 0.09838309
0
1
2
3
4
accuracy after epoch is 0.990099
corresponding loss 0.07913636
0
1
2
3
4
accuracy after epoch is 1.0
corresponding loss 0.06473966
0
1
2
3
4
accuracy after epoch is 1.0
corresponding loss 0.054127995
0
1
2
3
4
accuracy after epoch is 1.0
corresponding loss 0.04562374